In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/preprocessing/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main__fastscnn_hyperparameters_tuning'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [10]:
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.cloud.UNet import unet
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.PREPROCESSING_PATH

In [3]:
# === defining transforms ===

diameter = 7
sigma_color = 50
sigma_space = 100
horizontal_flip = custom_transforms.PartiallyDeterministicHorizontalFlip(p=0.5)
center_crop = custom_transforms.PartiallyDeterministicCenterCrop(p=0.5)

image_transform = T.Compose([
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# model parameters
model_name = "unet"
model = unet.UNet(out_channels=n_classes)
optimizer = optim.Adam

# setting up hyperparameters for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
loss_fn = nn.CrossEntropyLoss()
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

# Ray Tune parameters
cpus_per_trial = 0
gpus_per_trial = torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [6]:
# === training without augmentation ===

config = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "transform": "none",
    "from_checkpoint": False,
    "checkpoint_dir": local_dir
    }

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=config,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

2023-02-02 15:20:41,563	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-02 15:20:45,339	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (32 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


== Status ==
Current time: 2023-02-02 15:20:46 (running for 00:00:01.88)
Memory usage on this node: 4.3/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/preprocessing/UNet/train_model_2023-02-02_15-20-43
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+
| Trial name              | status   | loc               |
|-------------------------+----------+-------------------|
| train_model_290dc_00000 | RUNNING  | 172.28.0.12:72761 |
+-------------------------+----------+-------------------+




Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_290dc_00000,2023-02-02_16-22-17,True,,b07f328ad882460a91b33cab2359fb2b,0,50c3effd8f1c,10,172.28.0.12,72761,3684.98,367.48,3684.98,1675354937,0,,0.158486,0.548049,10,290dc_00000,0.170216,0.557993,0.00695562


== Status ==
Current time: 2023-02-02 15:30:50 (running for 00:10:06.16)
Memory usage on this node: 7.1/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/preprocessing/UNet/train_model_2023-02-02_15-20-43
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_290dc_00000 | RUNNING  | 172.28.0.12:72761 |     0.727117 |      0.328022 |   0.657559 |    0.349283 |                    1 |
+-------------------------+----------+-------------------+--

2023-02-02 16:22:18,106	INFO tune.py:762 -- Total run time: 3695.17 seconds (3693.80 seconds for the tuning loop).


== Status ==
Current time: 2023-02-02 16:22:17 (running for 01:01:33.80)
Memory usage on this node: 7.1/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/preprocessing/UNet/train_model_2023-02-02_15-20-43
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_290dc_00000 | TERMINATED | 172.28.0.12:72761 |     0.158486 |      0.548049 |   0.170216 |    0.557993 |                   10 |
+-------------------------+------------+-----------

In [ ]:
# === training with augmentation: ColorJitter ===

image_transform_with_augmentation = T.Compose([
    T.ColorJitter(brightness=0.25, contrast=0.25), 
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform)
    
model = unet.UNet(out_channels=n_classes)

config["transform"] = "color_jitter"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=config,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# === training with augmentation: horizontal_flip ===

image_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)
    
model = unet.UNet(out_channels=n_classes)

config["transform"] = "horizontal_flip"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=config,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# === training with augmentation: center_crop ===

image_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)

model = unet.UNet(out_channels=n_classes)

config["transform"] = "center_crop"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=config,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
%tensorboard --logdir {local_dir}